In [1]:
import pandas as pd
import numpy as np

#### Example of input data

In [2]:
#Read strata info table with columns:
#"Stratum": stratum ID, 1 - nstrata;
#"Nh": total number of units (5x5 km blocks) in each stratum;
#"Ah": stratum area (ha).
strata = pd.read_csv('Strata_info.txt', sep = '\t')

In [3]:
strata.head().style.hide(axis="index")

Stratum,Nh,Ah
1,1716105,4277431471.733470
2,925039,2305613901.934510
3,1335055,3327508028.205490
4,66438,165583367.781463


In [4]:
#Read sample interpretation table with columns:
#"Stratum": stratum ID, 1 - nstrata; 
#"Reference": proportion of target class (tree cover loss) from reference sample classification for each sample unit (5x5 km sample block);
#(optional)"Region": geographic region each sample unit (5x5 km block) belongs to.

data = pd.read_csv('Sample_data.txt', sep ='\t')
data.head().style.hide(axis="index")

ID,Stratum,Region,Reference
1,1,AFR,0.000000
2,1,AUS,0.000000
3,1,NAM,0.000000
4,1,LAM,0.000000
5,1,EUR,0.000000


In [7]:
#Merge data table with sample info table
data = data.merge(strata)
data.head().style.hide(axis="index")

ID,Stratum,Region,Reference,Nh,Ah
1,1,AFR,0.000000,1716105,4277431471.733470
2,1,AUS,0.000000,1716105,4277431471.733470
3,1,NAM,0.000000,1716105,4277431471.733470
4,1,LAM,0.000000,1716105,4277431471.733470
5,1,EUR,0.000000,1716105,4277431471.733470


#### Functions to estimate class area and its standard error

In [8]:
def estimate_area(df: pd.DataFrame) -> float:
    """ 
    Function to estimate target class area from sample refernce values 
    for sampling of units (pixels/polygons) with equal area.
    Strata weighted by their respective unit counts (Nh).
    ~~~
    Input dataframe with number of lines equal the number of sample pixels/polygons,
    and the following columns:
    "Stratum" (strata IDs 1 - nstrata)
    "Reference" (0-1 - proportion of the sample pixel/polygon identified as target class in reference classification;
    Reference data column could be defined as values in range from -1 to 1 to compute net change area of a target class,
    in this case negative proportions mean net loss, and positive proportions - net gain)
    "Ah" (stratum area, km²<or any other area unit>)
    "Nh" (number of units in each stratum h)
    ~~~
    Returns estimated target class area in units of Ah, 
    negative area in net change computations means overall net loss of a target class
    ~~~
    From Stehman (2014) "Estimating area and map accuracy for stratified random sampling when the strata are different from the map classes"
    https://doi.org/10.1080/01431161.2014.930207
    Equation 3
    """
    #Group input dataset by stratum
    ByStratum = df.groupby(by = ['Stratum'])
    
    #Equation 3
    Nh =  ByStratum.Nh.median()
    N = Nh.sum()
    proportionstratum = Nh * ByStratum.Reference.mean() / N
    proportiontotal = proportionstratum.sum()
    
    #Multiply estimated target class proportion by total area of the sampling region to convert the estimate into area units
    Atot = ByStratum.Ah.median().sum()
    area =  proportiontotal * Atot

    return area

In [9]:
def estimate_area_SE(df: pd.DataFrame) -> float:
    """ 
    Function to estimate target class area from sample refernce values 
    for sampling of units (pixels/polygons) with equal area.
    Strata weighted by their respective unit counts (Nh).
    ~~~
    Input dataframe with number of lines equal the number of sample pixels/polygons,
    and the following columns:
    "Stratum" (strata IDs 1 - nstrata)
    "Reference" (0-1 - proportion of the sample pixel/polygon identified as target class in reference classification;
    Reference data column could be defined as values in range from -1 to 1 to compute net change area of a target class,
    in this case negative proportions mean net loss, and positive proportions - net gain)
    "Ah" (stratum area, km²<or any other area unit>)
    "Nh" (number of units in each stratum h)
    ~~~
    Returns estimated SE of the target class area in in units of Ah,
    SE is always a positive number, even if the estimated target class area is negative
    ~~~
    From Stehman (2014) "Estimating area and map accuracy for stratified random sampling when the strata are different from the map classes"
    https://doi.org/10.1080/01431161.2014.930207
    Equations 25 and 26
    """
    
    ByStratum = df.groupby(by = ['Stratum'])

    #Equations 25 and 26
    Nh = ByStratum.Nh.median()
    nh = ByStratum.Reference.count()
    Forstrata = ByStratum.Reference.var(ddof=1) / nh
    StrataVar = Forstrata * (1 - nh / Nh) * Nh**2
    N = Nh.sum()
    StrataVarSum = StrataVar.sum() / N / N

    #Multiply estimated standard error of target class proportion by total area of the sampling region to convert the estimate into area units
    Atot = ByStratum.Ah.median().sum()
    SE = np.sqrt(StrataVarSum) * Atot
    
    return SE

In [10]:
#Estimate target class area (total tree cover loss area expressed in ha)
estimate_area(data)

27733433.49659045

In [11]:
#Estimate standard error of the target class area (SE of total tree cover loss area expressed in ha)
estimate_area_SE(data)

2195316.3073415556

In [12]:
#Estimate target class area for each Region

functions = [estimate_area, estimate_area_SE]
names = ['area, km²','area SE, km²']
results = pd.DataFrame()
datacopy = pd.DataFrame()

for classtype in data['Region'].unique():
    datacopy = data.copy()
    #A new reference column, where reference values are set to zero if class type is different from the current class type
    datacopy['Reference'] = np.where(datacopy['Region'] == classtype, datacopy['Reference'], 0)
    
    values = {}
    values = {nm:[fn(datacopy)] for fn, nm in zip(functions,names)}
    values["Estimate"] = classtype
    values_pd = pd.DataFrame(values).set_index("Estimate")
    results = pd.concat([values_pd, results])

results = results.reset_index()

results.style.hide(axis="index").format({name: '{:.4f}' for name in names})

Estimate,"area, km²","area SE, km²"
NEAS,4273056.8444,737573.5984
SEAS,5982019.1879,1732595.8486
EUR,1070554.6466,349671.5699
LAM,7390468.0232,1039215.8052
NAM,3406680.7657,853470.9224
AUS,575207.0389,247900.5206
AFR,5035446.9899,701011.7055


#### Functions to estimate % of class1 from class2

In [13]:
def estimate_class_percent (df: pd.DataFrame) -> float:
    """ 
    Function to estimate percent of class1 from class2, both estimated from the sample
    for sampling of units (pixels/polygons) with equal area.
    Strata weighted by their respective unit counts (Nh).
    ~~~ 
    Input dataframe with number of lines equal the number of sample pixels/polygons,
    and the following columns:
    "Stratum" (strata IDs 1 - nstrata)
    "Class1" (0-1 - proportion of the sample pixel/polygon identified as class 1 (percentage numerator))
    "Class2" (0-1 - proportion of the sample pixel/polygon identified as class 1 (percentage denumerator))
    "Nh" (number of units in each stratum h)
    ~~~
    Returns estimated percent of class1 from the area of class2
    ~~~
    From Stehman (2014) "Estimating area and map accuracy for stratified random sampling when the strata are different from the map classes"
    https://doi.org/10.1080/01431161.2014.930207
    Equation 27
    """
    ByStratum = df.groupby(by = ['Stratum'])
    
    #Equation 9 (with result expressed as percentage, i.e., estimated proportion (0-1) is multiplied by 100)
    Nh = ByStratum.Nh.median()
    Yest = (Nh * ByStratum.Class1.mean()).sum()
    Xest = (Nh * ByStratum.Class2.mean()).sum()
    PERC = Yest / Xest * 100

    return PERC

In [14]:
def estimate_class_percent_SE(df: pd.DataFrame) -> float:
    """ 
    Function to estimate the standard error (SE) of percent of class1 from class2, both estimated from the sample
    for sampling of units (pixels/polygons) with equal area.
    Strata weighted by their respective unit counts (Nh).
    ~~~
    Input dataframe with number of lines equal the number of sample pixels/polygons,
    and the following columns:
    "Stratum" (strata IDs 1 - nstrata)
    "Class1" (0-1 - proportion of the sample pixel/polygon identified as class 1 (percentage numerator))
    "Class2" (0-1 - proportion of the sample pixel/polygon identified as class 1 (percentage denumerator))
    "Nh" (number of units in each stratum h)
    ~~~
    Returns estimated SE of percent of class1 from class2
    ~~~
    From Stehman (2014) "Estimating area and map accuracy for stratified random sampling when the strata are different from the map classes"
    https://doi.org/10.1080/01431161.2014.930207
    Equations 27-29
    """
    df1 = df.copy()
    
    df1['XY'] = df1['Class1'] * df1['Class2']
    
    ByStratum = df1.groupby(by = ['Stratum'])

    #Equation 27
    Nh = ByStratum.Nh.median()
    Xest = (Nh * ByStratum.Class2.mean()).sum()
    Yest = (Nh * ByStratum.Class1.mean()).sum()
    R = Yest / Xest

    # Equations 28-29
    nh = ByStratum.Class1.count()
    meanyh = ByStratum.Class1.mean()
    meanxh = ByStratum.Class2.mean()
    vary = ByStratum.Class1.var(ddof=1)
    varx = ByStratum.Class2.var(ddof=1)
    covarxy = (ByStratum.XY.sum() - nh * meanyh * meanxh) / (nh - 1)
    StrataVar = Nh**2 * (1 - nh / Nh) * (vary + R**2 * varx - 2 * R * covarxy) / nh
    StrataVarSum = StrataVar.sum() / Xest / Xest

    #Convert estimated variance into standard error and express as percentage
    SE = np.sqrt(StrataVarSum) * 100
    
    return SE

In [15]:
#Example of use: estimate % of tree cover loss in each geographic region from the total area of tree cover loss (Reference>0).

functions = [estimate_class_percent, estimate_class_percent_SE]
names = ['% from target class','SE']
results=pd.DataFrame()
datacopy=pd.DataFrame()

for classtype in data['Region'].unique():
    datacopy = data.copy()
    #A new Class1 column, where reference values are set to zero if class type is different from the current class type
    datacopy['Class1']=np.where(datacopy['Region'] == classtype, datacopy['Reference'], 0)
    
    #Set Class2 equal to Reference (total area of target class)
    datacopy['Class2']=datacopy['Reference']
    
    values={}
    values = {nm:[fn(datacopy)] for fn, nm in zip(functions,names)}
    values["Estimate"]=classtype
    values_pd = pd.DataFrame(values).set_index("Estimate")
    results = pd.concat([values_pd, results])

results=results.reset_index()

results.style.hide(axis="index").format({name: '{:.2f}' for name in names})

Estimate,% from target class,SE
NEAS,15.41,2.66
SEAS,21.57,5.15
EUR,3.86,1.27
LAM,26.65,3.61
NAM,12.28,2.93
AUS,2.07,0.90
AFR,18.16,2.69
